# Compiling and Submitting Circuits onto Hilbert via Qiskit


<a target="_blank" href="https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/cq/cq_compile_qss.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>
<a target="_blank" href="https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/cq/cq_compile_qss.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder" align="left" style="margin-left: 10px;"/>
</a>

## Import Requirements

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the ```qiskit-superstaq``` client. You can try it out by running ```pip install qiskit-superstaq```:

In [ ]:
%pip install qiskit-superstaq

In [1]:
# Required imports
import qiskit
import qiskit_superstaq as qss

# Optional imports
import os  # used if setting a token as an environment variable

To interface Superstaq via Qiskit, we must first instantiate a service provider in ```qiskit-superstaq```. We then supply a Superstaq API key (which you can get from https://superstaq.super.tech.) by either providing the API key as an argument, i.e. : ```qss.superstaq_provider.SuperstaqProvider(api_key="token")``` , or by setting it as an environment variable. (see more details [here](https://docs-superstaq.readthedocs.io/en/stable/get_started/basics/basics_css.html#Set-up-access-to-Superstaq's-API)) 

In [2]:
provider = qss.superstaq_provider.SuperstaqProvider()

## Create a Circuit

First, we will create an example Qiskit circuit that we can then compile and submit onto Hilbert

In [3]:
circuit1 = qiskit.QuantumCircuit(1, 1)

circuit1.h(0)
circuit1.measure(0, 0)
circuit1.draw(fold=-1)

┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0

## Single Circuit Compilation

In [4]:
compiler_output = provider.cq_compile(circuit1)

In [5]:
print(compiler_output.circuit)

      ┌─────────────────┐┌─┐
 q_0: ┤ GR(1.57, -1.57) ├┤M├
      └─────────────────┘└╥┘
 q_1: ────────────────────╫─
                          ║ 
 q_2: ────────────────────╫─
                          ║ 
 q_3: ────────────────────╫─
                          ║ 
 q_4: ────────────────────╫─
                          ║ 
 q_5: ────────────────────╫─
                          ║ 
 q_6: ────────────────────╫─
                          ║ 
 q_7: ────────────────────╫─
                          ║ 
 q_8: ────────────────────╫─
                          ║ 
 q_9: ────────────────────╫─
                          ║ 
q_10: ────────────────────╫─
                          ║ 
q_11: ────────────────────╫─
                          ║ 
q_12: ────────────────────╫─
                          ║ 
q_13: ────────────────────╫─
                          ║ 
q_14: ────────────────────╫─
                          ║ 
q_15: ────────────────────╫─
                          ║ 
q_16: ────────────────────╫─
              

If you would like to compile (or submit) on a different number of qubits, this can be done via the ```grid_shape``` option. This simply sets the shape of the rectangular qubit grid. However, specifying a grid that is incompatable with Hilbert's current capabilities will result in an error. 

In [6]:
new_compiler_output = provider.cq_compile(circuit1, options={"grid_shape": (4, 6)})
print(new_compiler_output.circuit)

      ┌─────────────────┐┌─┐
 q_0: ┤ GR(1.57, -1.57) ├┤M├
      └─────────────────┘└╥┘
 q_1: ────────────────────╫─
                          ║ 
 q_2: ────────────────────╫─
                          ║ 
 q_3: ────────────────────╫─
                          ║ 
 q_4: ────────────────────╫─
                          ║ 
 q_5: ────────────────────╫─
                          ║ 
 q_6: ────────────────────╫─
                          ║ 
 q_7: ────────────────────╫─
                          ║ 
 q_8: ────────────────────╫─
                          ║ 
 q_9: ────────────────────╫─
                          ║ 
q_10: ────────────────────╫─
                          ║ 
q_11: ────────────────────╫─
                          ║ 
q_12: ────────────────────╫─
                          ║ 
q_13: ────────────────────╫─
                          ║ 
q_14: ────────────────────╫─
                          ║ 
q_15: ────────────────────╫─
                          ║ 
q_16: ────────────────────╫─
              

## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in backend from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [7]:
backend = provider.get_backend("cq_hilbert_simulator")
job = backend.run(circuit1, shots=100)
print(job.result().get_counts())

{'0': 55, '1': 45}


## Multiple circuit compilation

In [8]:
circuit2 = qiskit.QuantumCircuit(2)
circuit2.cnot(0, 1)
circuit2.h(1)
circuit2.draw(fold=-1)

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

In [9]:
compiler_output = provider.cq_compile([circuit1, circuit2])

In [10]:
print(compiler_output.circuits[0])
print()
print(compiler_output.circuits[1])

      ┌─────────────────┐┌─┐
 q_0: ┤ GR(1.57, -1.57) ├┤M├
      └─────────────────┘└╥┘
 q_1: ────────────────────╫─
                          ║ 
 q_2: ────────────────────╫─
                          ║ 
 q_3: ────────────────────╫─
                          ║ 
 q_4: ────────────────────╫─
                          ║ 
 q_5: ────────────────────╫─
                          ║ 
 q_6: ────────────────────╫─
                          ║ 
 q_7: ────────────────────╫─
                          ║ 
 q_8: ────────────────────╫─
                          ║ 
 q_9: ────────────────────╫─
                          ║ 
q_10: ────────────────────╫─
                          ║ 
q_11: ────────────────────╫─
                          ║ 
q_12: ────────────────────╫─
                          ║ 
q_13: ────────────────────╫─
                          ║ 
q_14: ────────────────────╫─
                          ║ 
q_15: ────────────────────╫─
                          ║ 
q_16: ────────────────────╫─
              